In [ ]:
import os, glob, re
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from scipy import ndimage

import face, private

# Refer to TCIA UC Davis healthy subject dataset for sample images
# glob should return a list of the dicom directories for CT or PET images
ct_paths = glob.glob(private.ct_paths)
pt_paths = glob.glob(private.pt_paths)

print(f'Found {len(ct_paths)} CT images and {len(pt_paths)} PET images')

# select subject 1, 90 minute timepoint
ct = [ct for ct in ct_paths if re.match(r'.*Sub001.*90m.*', ct)][0]
pt = [pt for pt in pt_paths if re.match(r'.*Sub001.*90m.*', pt)][0]

## CT Workflow

In [ ]:
# Load the CT image

nimg, _ = face.load_dicom(ct) # loads the dicom dir as a nifti image in memory
d = nimg.get_fdata()

In [ ]:
# Create the CT threshold image

middle = int(d.shape[2]/2)
slc = d[...,middle]
smax = slc.max()
slc = (slc / smax * 255.0).astype(np.uint8)
thr, mask = cv.threshold(slc, 0, 255,
                        cv.THRESH_BINARY + cv.THRESH_OTSU)

fig, axs = plt.subplots(ncols = 2)
axs[0].imshow(np.flipud(slc.T), cmap = 'gray')
axs[1].imshow(np.flipud(mask.T))
plt.show()

In [ ]:
# Create the offset imgae

d_mask = d > (thr / 255 * smax)
offset_unscaled = d_mask[:,::-1].argmax(1)
zooms = nimg.header.get_zooms()
aspect = zooms[2]/zooms[0]
pixdim = zooms[0]

offset = ndimage.zoom(offset_unscaled, (1,aspect), order = 1)

plt.imshow(offset.T, cmap = 'gray')
plt.gca().invert_yaxis()
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# Create the gradient image and the inverted 'rendering'

gradx = ndimage.sobel(offset, 0)
grady = ndimage.sobel(offset, 1)
grad = np.hypot(gradx, grady)

gmax = pixdim*20
grad = np.clip(grad, -gmax, gmax)

render = 1 - (grad / grad.max())
render = (render * 255).astype(np.uint8)

fig, axs = plt.subplots(ncols = 2)

axs[0].imshow(grad.T, cmap = 'gray')
axs[1].imshow(render.T, cmap = 'gray')

[a.invert_yaxis() for a in axs]
plt.show()

In [ ]:
# Locate the face in the image, and create a blurred (2d) replacement
# Note that this is somewhat different from what's actually performed
# for the 3d data.

l,b,w,h = face.find_face(render.T, pixdim, False)
pix = 8
w, h = w - w%pix, h - h%pix
r, t = l + w, b + h
rendert = render.T.copy()
rdr = rendert[b:t,l:r]
rdr_pix = ndimage.zoom(rdr, 1/pix)
rdr_pix = ndimage.zoom(rdr_pix, pix, order = 1)

fig, axs = plt.subplots(ncols = 2)

axs[0].imshow(rdr, cmap = 'gray')
axs[1].imshow(rdr_pix, cmap = 'gray')

[a.invert_yaxis() for a in axs]
plt.show()

In [ ]:
# Putting it all together, this runs the complete workflow on the CT image
# and shows the final anonymized CT nifti image

_ = face.anon_image(ct, plot = True)